In [1]:
import torch
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification

from trl import SFTTrainer, RewardTrainer

[2023-09-03 20:39:50,476] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
quantization_config = BitsAndBytesConfig(
     load_in_4bit=True,
)

In [3]:
torch_dtype = torch.bfloat16

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "gpt2",
    trust_remote_code=True,
    torch_dtype=torch_dtype
) 

In [20]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [21]:
dataset = load_dataset("Anthropic/hh-rlhf", split="train")

In [22]:
peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        bias="none",
        task_type="CAUSAL_LM",
    )

In [23]:
training_args = TrainingArguments(
    output_dir="sft",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    num_train_epochs=100,
    max_steps=100,
    
)

In [24]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    max_seq_length=512,
    train_dataset=dataset,
    dataset_text_field="chosen",
    #peft_config=peft_config,
)

Using pad_token, but it is not set yet.


In [25]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=100, training_loss=2.4484765625, metrics={'train_runtime': 78.0405, 'train_samples_per_second': 82.009, 'train_steps_per_second': 1.281, 'total_flos': 1580653486080000.0, 'train_loss': 2.4484765625, 'epoch': 0.04})

In [26]:
trainer.save_model("sft")

In [27]:
def preprocess_function(examples):
    new_examples = {
        "input_ids_chosen": [],
        "attention_mask_chosen": [],
        "input_ids_rejected": [],
        "attention_mask_rejected": [],
    }
    for chosen, rejected in zip(examples["chosen"], examples["rejected"]):
        tokenized_j = tokenizer(chosen, truncation=True)
        tokenized_k = tokenizer(rejected, truncation=True)

        new_examples["input_ids_chosen"].append(tokenized_j["input_ids"])
        new_examples["attention_mask_chosen"].append(tokenized_j["attention_mask"])
        new_examples["input_ids_rejected"].append(tokenized_k["input_ids"])
        new_examples["attention_mask_rejected"].append(tokenized_k["attention_mask"])

    return new_examples

In [28]:
reward_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/160800 [00:00<?, ? examples/s]

In [29]:
reward_model = AutoModelForSequenceClassification.from_pretrained(
    "sft",
    num_labels=1,
    torch_dtype=torch_dtype,
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at sft and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
reward_model.config.pad_token_id = tokenizer.eos_token_id

In [35]:
reward_model.config.use_cache = False

In [39]:
reward_training_args = TrainingArguments(
    output_dir="reward",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    num_train_epochs=100,
    max_steps=100,
    remove_unused_columns=False,
)

In [37]:
reward_trainer = RewardTrainer(
    model=reward_model,
    tokenizer=tokenizer,
    args=reward_training_args,
    train_dataset=reward_dataset,
    peft_config=peft_config,
)

/home/kashif/.env/pytorch/lib/python3.10/site-packages/trl/trainer/reward_trainer.py:123: UserWarning: When using RewardDataCollatorWithPadding, you should set `max_length` in the RewardTrainer's init it will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(


In [38]:
reward_trainer.train()

/mnt/scratch/kashif/transformers/src/transformers/tokenization_utils_base.py:2436: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss


TrainOutput(global_step=100, training_loss=1.2066622924804689, metrics={'train_runtime': 73.7294, 'train_samples_per_second': 21.701, 'train_steps_per_second': 1.356, 'total_flos': 0.0, 'train_loss': 1.2066622924804689, 'epoch': 0.01})

In [40]:
reward_trainer.save_model("reward")

In [78]:
def augment_score(examples):
    outputs = []
    for chosen, rejected in zip(examples["chosen"], examples["rejected"]):
        new_examples = {}
        tokenized_j = tokenizer(chosen, truncation=True, return_tensors="pt")
        tokenized_k = tokenizer(rejected, truncation=True, return_tensors="pt")

        new_examples["input_ids_chosen"] = tokenized_j["input_ids"]
        new_examples["attention_mask_chosen"] = tokenized_j["attention_mask"]
        new_examples["input_ids_rejected"]= tokenized_k["input_ids"]
        new_examples["attention_mask_rejected"]= tokenized_k["attention_mask"]

        reward_model.eval()
        with torch.no_grad():
            rewards_chosen = reward_model(
                input_ids=new_examples["input_ids_chosen"].to(reward_model.device),
                attention_mask=new_examples["attention_mask_chosen"].to(reward_model.device),
            )[0]
        
        outputs["rewards_chosen"] = rewards_chosen.float().cpu().numpy().tolist()
        
    return {'data': outputs}

In [79]:
reward_dataset.map(
    augment_score,
    batched=True,
    batch_size=8)

Map:   0%|          | 0/160800 [00:00<?, ? examples/s]

TypeError: list indices must be integers or slices, not str

In [75]:
%debug

> /tmp/ipykernel_12592/907666662.py(20)augment_score()
     18             )[0]
     19 
---> 20         outputs["rewards_chosen"] = rewards_chosen.float().cpu().numpy()
     21 
     22     return {'data': outputs}



ipdb>  rewards_chosen.float().cpu().numpy()


array([[2.]], dtype=float32)


ipdb>  rewards_chosen.float().cpu().numpy().tolist()


[[2.0]]


ipdb>  exit()


In [48]:
dataset[0]

{'chosen': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,\n\nHuman: What's your favorite one?\n\nAssistant: I haven't even thought about it.",
 'rejected': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, ji